---
title: "Gradient descent for linear regression"
format: html
---

In [1]:
import Pkg; Pkg.activate(".")
using DataFrames
import CSV
using CairoMakie
using Statistics
CairoMakie.activate!(; px_per_unit = 2)

ArgumentError: ArgumentError: Package DataFrames not found in current path.
- Run `import Pkg; Pkg.add("DataFrames")` to install the DataFrames package.

In [ ]:
# Load the data
data = 